In [1]:
import game
import player
import environment
import parsers
import runner
import numpy as np
import pandas as pd
from importlib import *

In [2]:
values = [(1, 1), (2, 2), (3, 3)]
targets = range(len(values))
max(targets, key=lambda x: values[x][0])

2

let's initialize a game with a Stackelberg attacker and a Uniform defender

In [3]:
g = game.Game(((1, 1), (2, 2), (3, 3)), 10)
d = player.Defender(g, 0, 1)
a = player.StackelbergAttacker(g, 1, 1)
g.players[0] = d
g.players[1] = a
g.defenders.append(0)
g.attackers.append(1)
e = environment.Environment(g, 0)
agent = d

we can now play the game

In [4]:
for t in range(g.time_horizon):
    strategy = agent.compute_strategy()
    e.observe_strategy(strategy)
    realization = agent.sample_strategy()
    e.observe_realization(realization)
    feedback = e.feedback("expert")
    agent.receive_feedback(feedback)

In [5]:
agent.feedbacks

[{0: 0, 1: 0, 2: 0},
 {0: 0, 1: 0, 2: 0},
 {0: 0, 1: 0, 2: -3},
 {0: 0, 1: 0, 2: -3},
 {0: 0, 1: 0, 2: -3},
 {0: 0, 1: 0, 2: 0},
 {0: 0, 1: 0, 2: -3},
 {0: 0, 1: 0, 2: -3},
 {0: 0, 1: 0, 2: -3},
 {0: 0, 1: 0, 2: -3}]

and here is how it has gone

In [6]:
g.history

[{0: [2], 1: [2]},
 {0: [2], 1: [2]},
 {0: [1], 1: [2]},
 {0: [0], 1: [2]},
 {0: [1], 1: [2]},
 {0: [2], 1: [2]},
 {0: [1], 1: [2]},
 {0: [1], 1: [2]},
 {0: [1], 1: [2]},
 {0: [1], 1: [2]}]

In [7]:
g.strategy_history

[{0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]},
 {0: [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
  1: [0, 0, 1]}]

## Configuration Files

we can also set a configuration file as below

In [8]:
df = pd.read_csv("conf.csv")

In [9]:
df

,T,0,1,2,3,4,Defender,Attacker,Attacker.1
0,5,2,1,3,1,1,defender,player,stackelberg
1,3,1,1,1,1,1,defender,NaN,stackelberg
2,6,1,3,1,2,1,stu_defender,stackelberg,NaN
3,5,1,12,13,1,1,defender,player,stackelberg


In [10]:
mypath = "/home/lorenzo/Scrivania/Polimi/Thesis/code/onlineSG/games"

and use the *Runner* class to run all the configuration files in a folder

In [20]:
r = runner.Runner(mypath)

In [21]:
r.run()

In [23]:
r.batches[0].experiments[2].game.strategy_history

[{0: [0, 1, 0, 0, 0], 1: [0, 0, 0, 1, 0]},
 {0: [0, 1, 0, 0, 0], 1: [0, 0, 0, 1, 0]},
 {0: [0, 1, 0, 0, 0], 1: [0, 0, 0, 1, 0]},
 {0: [0, 1, 0, 0, 0], 1: [0, 0, 0, 1, 0]},
 {0: [0, 1, 0, 0, 0], 1: [0, 0, 0, 1, 0]},
 {0: [0, 1, 0, 0, 0], 1: [0, 0, 0, 1, 0]}]

In [24]:
r.batches[0].experiments[2].agent.feedbacks

[{0: 0, 1: 0, 2: 0, 3: -2, 4: 0},
 {0: 0, 1: 0, 2: 0, 3: -2, 4: 0},
 {0: 0, 1: 0, 2: 0, 3: -2, 4: 0},
 {0: 0, 1: 0, 2: 0, 3: -2, 4: 0},
 {0: 0, 1: 0, 2: 0, 3: -2, 4: 0},
 {0: 0, 1: 0, 2: 0, 3: -2, 4: 0}]

In [25]:
r.batches[0].experiments[2].game.history

[{0: [1], 1: [3]},
 {0: [1], 1: [3]},
 {0: [1], 1: [3]},
 {0: [1], 1: [3]},
 {0: [1], 1: [3]},
 {0: [1], 1: [3]}]

In [26]:
r.batches[0].experiments[2].agent

In [27]:
# if you change something, don't forget to reload

reload(player)
reload(parsers)
reload(game)
reload(environment)
reload(runner)

<module 'runner' from '/home/lorenzo/Scrivania/Polimi/Thesis/code/onlineSG/runner.py'>

In [28]:
from gurobipy import *

In [88]:
m = Model()

In [89]:
v0 = m.addVar()

In [90]:
v1 = m.addVar()

In [91]:
m.update()

In [92]:
m.addConstr(v0-v1 <= 4)
m.addConstr(v0+v1 <= 4)
m.addConstr(-0.25*v0+v1 <= 1)
m.setObjective(v1, GRB.MAXIMIZE)

In [93]:
m.params.outputflag = 0

In [94]:
m.optimize()

In [97]:
import matplotlib.pyplot as pyplot
pyplot.plot([0,4], [0,4])
pyplot.plot([4,0], [0,4])
pyplot.plot([0,4], [1,2])
pyplot.plot([v0.x], [v1.x], 'ro')
pyplot.show()

In [96]:
for v in m.getVars():
    print(v.varName, v.x)

C0 2.4
C1 1.6


In [75]:
help(m.params.outputflag)

int(x=0) -> integer
int(x, base=10) -> integer

Convert a number or string to an integer, or return 0 if no arguments
are given.  If x is a number, return x.__int__().  For floating point
numbers, this truncates towards zero.

If x is not a number or if base is given, then x must be a string,
bytes, or bytearray instance representing an integer literal in the
given base.  The literal can be preceded by '+' or '-' and be surrounded
by whitespace.  The base defaults to 10.  Valid bases are 0 and 2-36.
Base 0 means to interpret the base from the string as an integer literal.
>>> int('0b100', base=0)
4


In [60]:
help(v0)


  Gurobi variable object.  Variables have a number of attributes.
  Some can be set (e.g., v.ub = 0.0), while others can only be queried
  (e.g., print v.x). The most commonly used variable attributes are:
    obj: Linear objective coefficient.
    lb: Lower bound.
    ub: Upper bound.
    varName: Variable name.
    vType: Variable type ('C', 'B', 'I', 'S', or 'N').
    x: Solution value.
    rc: Solution reduced cost.
    xn: Solution value in an alternate MIP solution.

  Type "help(GRB.attr)" for a list of all available attributes.

  Note that attribute modifications are handled in a lazy fashion.  You
  won't see the effect of a change until after the next call to Model.update()
  or Model.optimize().
  


In [74]:
v0

<gurobi.Var C0>

In [100]:
try:

    # Create a new model
    m = Model("glasses")

    # Create variables
    x = m.addVar(vtype=GRB.CONTINUOUS, name="x")
    y = m.addVar(vtype=GRB.CONTINUOUS, name="y")

    # Set objective
    m.setObjective(3 * x + 5 * y, GRB.MAXIMIZE)

    # Add constraint:
    m.addConstr(x <= 4, "c0")

    # Add constraint:
    m.addConstr(y <= 6, "c1")
    
    # Add constraint:
    m.addConstr(3 * x + 2 * y <= 18, "c2")

    m.optimize()

    for v in m.getVars():
        print(v.varName, v.x)

    print('Obj:', m.objVal)

except GurobiError:
    print('Error reported')


Optimize a model with 3 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2000000e+01   1.500000e+00   0.000000e+00      0s
       1    3.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds
Optimal objective  3.600000000e+01
x 2.0
y 6.0
Obj: 36.0
